<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import" data-toc-modified-id="Import-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import</a></span><ul class="toc-item"><li><span><a href="#Libraries" data-toc-modified-id="Libraries-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Libraries</a></span></li><li><span><a href="#Data-download" data-toc-modified-id="Data-download-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Data download</a></span></li></ul></li><li><span><a href="#Building-the-model" data-toc-modified-id="Building-the-model-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Building the model</a></span></li><li><span><a href="#Preparing-the-data" data-toc-modified-id="Preparing-the-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Preparing the data</a></span></li><li><span><a href="#Training-the-models" data-toc-modified-id="Training-the-models-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Training the models</a></span><ul class="toc-item"><li><span><a href="#Model-1:-Random-normal" data-toc-modified-id="Model-1:-Random-normal-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Model 1: Random normal</a></span></li><li><span><a href="#Model-2:-Uniform" data-toc-modified-id="Model-2:-Uniform-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Model 2: Uniform</a></span></li><li><span><a href="#Model-3:-He-normal" data-toc-modified-id="Model-3:-He-normal-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Model 3: He normal</a></span></li><li><span><a href="#Model-4:-Orthogonal" data-toc-modified-id="Model-4:-Orthogonal-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Model 4: Orthogonal</a></span></li><li><span><a href="#Model-5:-LeCun-Uniform" data-toc-modified-id="Model-5:-LeCun-Uniform-4.5"><span class="toc-item-num">4.5&nbsp;&nbsp;</span>Model 5: LeCun Uniform</a></span></li><li><span><a href="#Model-6:-Orthogonal" data-toc-modified-id="Model-6:-Orthogonal-4.6"><span class="toc-item-num">4.6&nbsp;&nbsp;</span>Model 6: Orthogonal</a></span></li></ul></li><li><span><a href="#Evaluation" data-toc-modified-id="Evaluation-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Evaluation</a></span><ul class="toc-item"><li><span><a href="#One-model-at-a-time" data-toc-modified-id="One-model-at-a-time-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>One model at a time</a></span></li><li><span><a href="#All-the-models-together" data-toc-modified-id="All-the-models-together-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>All the models together</a></span></li></ul></li></ul></div>

## Import

### Libraries 

In [ ]:
from tensorflow import keras
import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from tensorflow import convert_to_tensor
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.resnet50 import ResNet50
from keras import Input
from keras.models import load_model

In [ ]:
np.random.seed(813306)

### Data download

In [ ]:
with open('objs_train_label.pkl', 'rb') as f:
    images_2010_2, labels_2010_2 = pickle.load(f)

images_2010_2 = np.array(images_2010_2, dtype=np.uint8)

## Building the model

In [ ]:
def build_resnet(input_shape, n_feature_maps, nb_classes, initial):
    print ('build conv_x')
    x = keras.layers.Input(shape=(input_shape))
    conv_x = keras.layers.BatchNormalization()(x)
    conv_x = keras.layers.Conv2D(n_feature_maps, 8, 1, padding='same', kernel_initializer=initial)(conv_x)
    conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = keras.layers.Activation('relu')(conv_x)
     
    print ('build conv_y')
    conv_y = keras.layers.Conv2D(n_feature_maps, 5, 1, padding='same', kernel_initializer=initial)(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = keras.layers.Activation('relu')(conv_y)
     
    print ('build conv_z')
    conv_z = keras.layers.Conv2D(n_feature_maps, 3, 1, padding='same', kernel_initializer=initial)(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)
     
    is_expand_channels = not (input_shape[-1] == n_feature_maps)
    if is_expand_channels:
        shortcut_y = keras.layers.Conv2D(n_feature_maps, 1, 1,padding='same', kernel_initializer=initial)(x)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)
    else:
        shortcut_y = keras.layers.BatchNormalization()(x)
    print ('Merging skip connection')
    y = keras.layers.Add()([shortcut_y, conv_z])
    y = keras.layers.Activation('relu')(y)
     
    print ('build conv_x')
    x1 = y
    conv_x = keras.layers.Conv2D(n_feature_maps*2, 8, 1, padding='same', kernel_initializer=initial)(x1)
    conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = keras.layers.Activation('relu')(conv_x)
     
    print ('build conv_y')
    conv_y = keras.layers.Conv2D(n_feature_maps*2, 5, 1, padding='same', kernel_initializer=initial)(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = keras.layers.Activation('relu')(conv_y)
     
    print ('build conv_z')
    conv_z = keras.layers.Conv2D(n_feature_maps*2, 3, 1, padding='same', kernel_initializer=initial)(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)
     
    is_expand_channels = not (input_shape[-1] == n_feature_maps*2)
    if is_expand_channels:
        shortcut_y = keras.layers.Conv2D(n_feature_maps*2, 1, 1,padding='same', kernel_initializer=initial)(x1)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)
    else:
        shortcut_y = keras.layers.BatchNormalization()(x1)
    print ('Merging skip connection')
    y = keras.layers.Add()([shortcut_y, conv_z])
    y = keras.layers.Activation('relu')(y)
     
    print ('build conv_x')
    x1 = y
    conv_x = keras.layers.Conv2D(n_feature_maps*2, 8, 1, padding='same', kernel_initializer=initial)(x1)
    conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = keras.layers.Activation('relu')(conv_x)
     
    print ('build conv_y')
    conv_y = keras.layers.Conv2D(n_feature_maps*2, 5, 1, padding='same', kernel_initializer=initial)(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = keras.layers.Activation('relu')(conv_y)
     
    print ('build conv_z')
    conv_z = keras.layers.Conv2D(n_feature_maps*2, 3, 1, padding='same', kernel_initializer=initial)(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)

    is_expand_channels = not (input_shape[-1] == n_feature_maps*2)
    if is_expand_channels:
        shortcut_y = keras.layers.Conv2D(n_feature_maps*2, 1, 1,padding='same', kernel_initializer=initial)(x1)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)
    else:
        shortcut_y = keras.layers.BatchNormalization()(x1)
    print ('Merging skip connection')
    y = keras.layers.Add()([shortcut_y, conv_z])
    y = keras.layers.Activation('relu')(y)
     
    full = keras.layers.GlobalAveragePooling2D()(y)
    out = keras.layers.Dense(nb_classes, activation='softmax')(full)
    print ('        -- model was built.')
    return x, out


## Preparing the data

In [ ]:
imshow(images_2010_2[0])

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(images_2010_2, labels_2010_2, test_size=0.3)
nb_classes = 2
nb_epochs = 5
batch_size = 16

Y_train = keras.utils.to_categorical(y_train, nb_classes)
Y_test = keras.utils.to_categorical(y_test, nb_classes)

x_train_mean = x_train.mean()
x_train_std = x_train.std()
x_train = (x_train - x_train_mean)/(x_train_std)
x_test = (x_test - x_train_mean)/(x_train_std)

## Training the models

### Model 1: Random normal

In [ ]:
x1 , y1 = build_resnet(x_train.shape[1:], 64, nb_classes, 'random_normal')
model1 = keras.models.Model(inputs=x1, outputs=y1)
optimizer = keras.optimizers.Adam()
model1.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5,
                  patience=50, min_lr=0.0001) 
hist = model1.fit(x_train, Y_train, batch_size=batch_size, epochs=nb_epochs, verbose=1, validation_data=(x_test, Y_test), callbacks = [reduce_lr])
log = pd.DataFrame(hist.history)

In [ ]:
model1.save('model1_label.h5')

### Model 2: Uniform 

In [ ]:
x2 , y2 = build_resnet(x_train.shape[1:], 64, nb_classes, 'uniform')
model2 = keras.models.Model(inputs=x2, outputs=y2)
optimizer = keras.optimizers.Adam()
model2.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5,
                  patience=50, min_lr=0.0001) 
hist = model2.fit(x_train, Y_train, batch_size=batch_size, epochs=nb_epochs, verbose=1, validation_data=(x_test, Y_test), callbacks = [reduce_lr])
log = pd.DataFrame(hist.history)

In [ ]:
model2.save('model2_label.h5')

### Model 3: He normal

In [ ]:
x3 , y3 = build_resnet(x_train.shape[1:], 64, nb_classes, 'he_normal')
model3 = keras.models.Model(inputs=x3, outputs=y3)
optimizer = keras.optimizers.Adam()
model3.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5,
                  patience=50, min_lr=0.0001) 
hist = model3.fit(x_train, Y_train, batch_size=batch_size, epochs=nb_epochs, verbose=1, validation_data=(x_test, Y_test), callbacks = [reduce_lr])
log = pd.DataFrame(hist.history)

In [ ]:
model3.save('model3_label.h5')

### Model 4: Orthogonal

In [ ]:
x4 , y4 = build_resnet(x_train.shape[1:], 64, nb_classes, 'orthogonal')
model4 = keras.models.Model(inputs=x4, outputs=y4)
optimizer = keras.optimizers.Adam()
model4.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5,
                  patience=50, min_lr=0.0001) 
hist = model4.fit(x_train, Y_train, batch_size=batch_size, epochs=nb_epochs, verbose=1, validation_data=(x_test, Y_test), callbacks = [reduce_lr])
log = pd.DataFrame(hist.history)

In [ ]:
model4.save('model4_label.h5')

### Model 5: LeCun Uniform

In [ ]:
x5 , y5 = build_resnet(x_train.shape[1:], 64, nb_classes, 'lecun_uniform')
model5 = keras.models.Model(inputs=x5, outputs=y5)
optimizer = keras.optimizers.Adam()
model5.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5,
                  patience=50, min_lr=0.0001) 
hist = model5.fit(x_train, Y_train, batch_size=batch_size, epochs=nb_epochs, verbose=1, validation_data=(x_test, Y_test), callbacks = [reduce_lr])
log = pd.DataFrame(hist.history)

In [ ]:
model5.save('model5_label.h5')

## Evaluation

In [ ]:
model1 = load_model('model1_label.h5')

In [ ]:
model2 = load_model('model2_label.h5')

In [ ]:
model3 = load_model('model3_label.h5')

In [ ]:
model4 = load_model('model4_label.h5')

In [ ]:
model5 = load_model('model5_label.h5')

In [ ]:
with open('objs_sandp2.pkl', 'rb') as f:
    images_sp, values_sp = pickle.load(f)

In [ ]:
images_sp = (images_sp - x_train_mean)/(x_train_std)

In [ ]:
def one_model(testing_images, testing_values, x_mean, x_std, mod):
    TP = 0.0
    TN = 0.0
    FP = 0.0
    FN = 0.0
    iterations = len(testing_images)
#     investment = [1000.0 for i in range(iterations + 1)]
    investment = 1000.0
    nb_classes = 2
    nb_epochs = 2
    
    for i in range(iterations):
#         if i%1 == 0:
        if i%10 == 0:
#             print(str(i) + '-th iteration on ' + str(iterations) + '. Investment at ' + str(investment[i]) + '.')
            print(str(i) + '-th iteration on ' + str(iterations) + '.\n' + str(investment))
        prediction = mod.predict(np.array([testing_images[i]]))[0]
        predict = np.argmax(prediction)

        if prediction[predict] > 0.6:
#             investment[i+1] = investment[i] * (1 - prediction[predict]) +\
#                               investment[i] * prediction[predict] *\
#                               (testing_values[i][1]/testing_values[i][0] if (predict == 1) 
#                               else testing_values[i][0]/testing_values[i][1])
            investment = investment * (1 - prediction[predict]) +\
                         investment * prediction[predict] *\
                         (testing_values[i] if (predict == 0) 
                         else 1/testing_values[i])
    
        if predict == 1:
            if testing_values[i] >= 1: 
                TP += 1
            else: 
                FP += 1
        else:
            if testing_values[i] <= 1: 
                TN += 1
            else: 
                FN += 1
            
        with open('testing3/objs_' + str(i + 240) + '.pkl', 'rb') as f:
            images, labels = pickle.load(f)
        
        images = np.array(images, dtype=np.uint8)
            
        x_training, x_testing, y_training, y_testing = train_test_split(images, labels, test_size=0.3)
        batch_size = int(max(min(x_training.shape[0]/10, 16), 1))

        Y_training = keras.utils.to_categorical(y_training, nb_classes)
        Y_testing = keras.utils.to_categorical(y_testing, nb_classes)

        x_training = (x_training - x_mean)/(x_std)
        x_testing = (x_testing - x_mean)/(x_std)
        
        reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5,
                                                      patience=50, min_lr=0.0001) 
        
        mod.fit(x_training, Y_training, batch_size=batch_size, epochs=nb_epochs, verbose=0, validation_data=(x_testing, Y_testing), callbacks = [reduce_lr])
    return [TP, TN, FP, FN, investment]

In [ ]:
one_model(images_sp, values_sp, x_train_mean, x_train_std, resnet)